In [0]:
# =============================================================
# JOB 3 — INPE SILVER PROCESS
# =============================================================
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

spark = SparkSession.builder.appName("INPE_Silver_Process").getOrCreate()

TABELA_BRONZE = "datamasters.b_inep.focos_queimadas_stream"
TABELA_SILVER = "datamasters.s_inep.d_foco_queim_format_on"

print(f"⚙️ Processando dados: {TABELA_BRONZE} → {TABELA_SILVER}")

df_bronze = spark.table(TABELA_BRONZE)

df_silver = (
    df_bronze
    .withColumnRenamed("latitude", "lat")
    .withColumnRenamed("longitude", "lon")
    .withColumn("valido", when(col("lat").isNotNull() & col("lon").isNotNull(), True).otherwise(False))
    .filter(col("valido") == True)
    .dropDuplicates(["lat", "lon", "datahora"])
)

(
    df_silver.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(TABELA_SILVER)
)

print("✅ Silver atualizada com sucesso.")